Read data from file & Initialization

In [496]:
import glob
from math import log

hamfilenames = glob.glob('./data/ham/*.words')
spamfilenames = glob.glob('./data/spam/*.words')

hamdict = dict()
spamdict = dict()

totalhamwords = 0.0
totalspamwords = 0.0
totalwords = 0.0

logprobham = 0.0
logprobspam = 0.0

Building Model on Train data:
1. Calculating Priori's

In [497]:
hamfilecount = len(hamfilenames)
spamfilecount = len(spamfilenames)
totalfilecount = hamfilecount + spamfilecount

print('Total file count: '+str(totalfilecount))

hamprob = float(hamfilecount/totalfilecount)
spamprob = float(spamfilecount/totalfilecount)

logclassham = log(hamprob)
logclassspam = log(spamprob)

print('Log Ham prob: '+str(logclassham))
print('Log Spam prob: '+str(logclassspam))


Total file count: 1000
Log Ham prob: -0.6931471805599453
Log Spam prob: -0.6931471805599453


Building Model on Train data:
1. Calculating count of words per corresponding class 
2. Calculating total distinct words in the current corpus

In [498]:
for f in hamfilenames:
   # print(f)
    with open(f) as file:
        for line in file:
            line = line.lower().strip()
            # print(line)
            if line == "":
                continue
            hamdict[line] = hamdict.get(line, 0.0) + 1.0
            totalhamwords+=1

for f in spamfilenames:
   # print(f)
    with open(f) as file:
        for line in file:
            line = line.lower().strip()
            if line == "":
                continue
            # print(line)
            spamdict[line] =spamdict.get(line, 0.0) + 1.0
            totalspamwords+=1

distinctWords = dict()
distinctWords.update(hamdict)
distinctWords.update(spamdict)
totalDistinctwords=len(distinctWords)

Classification on Test data:

The algorithm calculates probablity of class given an email(list of words).This probablity is calculated by adding logarithmic probablity of classes(priori's) and the logarthmic probablity of word being a spam or ham(over all words)
a. The second part is calculated by dividing the count of a word being spam or ham from (spamdict or hamdict) and dividing it by total words in ham or spam classes(totalhamwords or totalspamwords).Both of these are calculted in the above section.
b.To the numerator, we add the alpha smoothening parameter and to the denominator we add alpha times the count of vocabulary.

1. Calculating Posteriori (probability of classes given per email)
2. Uses smoothing with |V| as total vocabulary close to english language. (This is like having each word in english language appear atleast once in every class)
3. Using alpha as a smoothing parameter.


Tested using 
a. Vocabulary of current corpus
b. Vocabulary of English language(total english dictionary words)
c. Changing alpha from 0.0001 to 1000


In [522]:
testfilenames =  glob.glob('./data/test/*.words')
testfiletruth =  glob.glob('./data/test/truthfile')
result = ['Spam' for i in range(0, 100)]
truth = ['Spam' for i in range(0, 100)]

with open('./data/test/truthfile') as file:
    for line in file:
        line = line.strip()
        truth[int(line) - 1] = 'Ham'

alpha=0.001
totalengwords = 171476 #totalDistinctwords # 171476

for f in testfilenames: 
    logprobham = 0.0
    logprobspam = 0.0
    
    with open(f) as file:
        name = (file.name.split('\\')[1]).split('.')[0]
        for line in file:
            line = line.lower().strip()
            if line == "":
                continue
            wordcountham = hamdict.get(line, 0)
            probham = float(wordcountham + 1.0*alpha)/float(hamfilecount + totalengwords*alpha)
            logprobham += log(probham)
        
            wordcountspam = spamdict.get(line, 0)
            probspam = float(wordcountspam + 1.0*alpha)/float(spamfilecount + totalengwords*alpha)
            logprobspam += log(probspam)

        logprobham += logclassham
        logprobspam += logclassspam

        if logprobham <= logprobspam:
            result[int(name) - 1] = 'Ham'
        else:
            result[int(name) - 1] = 'Spam'


Testing data against its label

In [523]:
count=0;
for i in range(0, 100):
    print(i, result[i], truth[i])
    if result[i] == truth[i]:
        count=count+1
    
print(count)
from sklearn.metrics import classification_report
print(classification_report(truth, result, target_names=['Ham', 'Spam']))

0 Ham Ham
1 Spam Spam
2 Spam Spam
3 Spam Spam
4 Ham Ham
5 Spam Spam
6 Spam Spam
7 Spam Spam
8 Spam Spam
9 Ham Ham
10 Spam Spam
11 Spam Spam
12 Ham Ham
13 Ham Ham
14 Ham Ham
15 Spam Spam
16 Ham Ham
17 Ham Ham
18 Ham Ham
19 Ham Ham
20 Ham Ham
21 Spam Spam
22 Ham Ham
23 Ham Ham
24 Ham Ham
25 Spam Spam
26 Spam Spam
27 Ham Ham
28 Spam Spam
29 Spam Spam
30 Ham Ham
31 Ham Ham
32 Spam Spam
33 Spam Spam
34 Ham Ham
35 Ham Ham
36 Ham Ham
37 Ham Ham
38 Spam Spam
39 Ham Ham
40 Spam Spam
41 Ham Ham
42 Ham Ham
43 Ham Ham
44 Ham Ham
45 Ham Ham
46 Spam Spam
47 Spam Spam
48 Spam Spam
49 Ham Ham
50 Ham Ham
51 Spam Spam
52 Spam Spam
53 Spam Spam
54 Ham Ham
55 Spam Spam
56 Spam Spam
57 Ham Ham
58 Spam Spam
59 Spam Spam
60 Spam Spam
61 Spam Spam
62 Ham Ham
63 Spam Spam
64 Ham Ham
65 Ham Ham
66 Spam Spam
67 Ham Ham
68 Ham Spam
69 Ham Spam
70 Ham Spam
71 Ham Spam
72 Ham Ham
73 Ham Spam
74 Ham Spam
75 Ham Spam
76 Spam Spam
77 Spam Spam
78 Spam Spam
79 Spam Spam
80 Ham Spam
81 Ham Spam
82 Ham Spam
83 Ham Spam
8